In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/scraped_data/reviews_Electronics_5.json.gz')

In [2]:
df.shape

(1689188, 9)

In [3]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [4]:
df['reviewText_len'] = df['reviewText'].apply(lambda x:len(x))

In [5]:
df['reviewText_len'].mean()

634.201737166023

In [6]:
g = df.groupby('overall')

In [7]:
df['asin'].nunique()

63001

In [8]:
for name, group in g:
    print (name,(group['reviewText_len']).count())

1.0 108725
2.0 82139
3.0 142257
4.0 347041
5.0 1009026


In [10]:
df_meta = getDF('data/scraped_data/meta_Electronics.json.gz')

In [11]:
df_meta.head(3)

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN


In [12]:
df_comb = df.merge?

In [ ]:
df_comb = df.merge

In [13]:
df_comb = df.merge(df_meta, how="left", on="asin")

In [14]:
df_comb.shape

(1689188, 18)

In [15]:
df_comb.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewText_len,imUrl,description,categories,title,price,salesRank,related,brand
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013",805,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010",2175,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010",4607,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [16]:
df_sample = df_comb[df_comb['categories'].apply(lambda x:'tv' in (str(x)).lower())]
#df_sample = df_sample[df_sample['brand'].apply(lambda x:'samsung' in (str(x)).lower())]

In [17]:
df_prod = df_sample[['overall','reviewText','title','brand','categories']]

In [18]:
df_prod = df_prod.rename(columns={'overall':'rating', 'reviewText':'reviews','title':'product','categories':'category'})
df_prod.count()

rating      64262
reviews     64262
product     63611
brand       31268
category    64262
dtype: int64

In [19]:
from src.naive_review_analyzer import NaiveReviewAnalyzer
nra = NaiveReviewAnalyzer()
nra.create_bow(df_prod)

for name, group in df_prod.groupby('product'):
    print(name, '\n\t',nra.create_word_list(product=name, rating=[5.0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/Users/bhaskarballapragada/Documents/Galvanize/product_review_sentiments/src/naive_review_analyzer.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_prod = df_prod[self.df['rating'].apply(lambda x: x in rating)]
/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


1-Port Bi-Directional Cable TV HDTV Amplifier Splitter Signal Booster with Passive Return 
	 ([('amp', 100), ('signal', 99), ('port', 95), ('booster', 91), ('antenna', 85), ('air', 81), ('reg', 80), ('strenthen', 80), ('hd', 80), ('item', 79), ('power', 79), ('unit', 78), ('strength', 78), ('pct', 77), ('quality', 77), ('splitter', 75), ('warner', 75), ('feet', 75), ('channels', 74), ('boost', 73), ('brick', 73), ('rca', 72), ('amplifier', 71), ('issue', 71), ('bedroom', 71), ('output', 71), ('mini', 71), ('rating', 70), ('split', 70)], [('hot', 80), ('digital', 80), ('cool', 80), ('excellent', 79), ('fine', 75), ('short', 72)])
10&quot; to 24&quot; Universal Tilt Wall Mount 
	 ([('monitor', 100), ('tilt', 97), ('wall', 91), ('degrees', 85), ('screw', 85), ('computer', 85), ('invert', 85), ('monitors', 85), ('option', 84), ('laptop', 76), ('desk', 76), ('bang', 74), ('dollar', 74), ('buck', 74), ('mind', 74), ('cost', 74), ('trade', 74), ('addition', 74), ('problem', 74), ('quality', 7

4-Way Coax Cable Splitter 2Ghz Cable TV Video HDTV 
	 ([('job', 100), ('dose', 100), ('splitter', 95), ('split', 94), ('cable', 83), ('loss', 77), ('issues', 77), ('connections', 75), ('blu', 75), ('vcr', 75), ('ray', 75), ('cd', 75), ('antenna', 72), ('degradation', 65), ('broadcasts', 65), ('conjunction', 65), ('dvr', 65), ('anything', 65), ('item', 65), ('rooms', 65), ('tvs', 65), ('receivers', 65), ('rca', 65), ('quality', 65)], [('sure', 77), ('separate', 75), ('different', 65), ('fine', 65), ('worth', 65), ('terrestrial', 65), ('outdoor', 65)])
40, 42, 43 inch TV-ProtectorTM Stalysh TV Screen Protector for LCD, LED and Plasma TVs 
	 ([('product', 100), ('television', 99), ('year', 96), ('things', 95), ('velcro', 93), ('price', 91), ('edge', 89), ('issue', 88), ('adhesive', 88), ('lcd', 86), ('shipping', 86), ('saturday', 86), ('pets', 86), ('wii', 86), ('children', 86), ('cat', 86), ('wednesday', 86)], [('good', 99), ('old', 96), ('thick', 96), ('transparent', 96), ('expensive', 

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [29]:
import ipywidgets
import pyLDAvis.sklearn
from src.LdaReviewAnalyzer import LdaReviewAnalyzer
lda = LdaReviewAnalyzer(num_topics=5)
lda.build_vectorize(df_prod)
lda.fit(df_prod, rating=[5])
lda.transform()
topics_dict = lda.get_topics()
#lda.display_topic_model()

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [30]:
print (topics_dict)

{'Topic 0:': [('would', 5300.158212403265), ('like', 5282.252313740159), ('good', 5045.357059550335), ('price', 4672.704170510319), ('get', 4427.02868046432), ('really', 4051.7575958682855), ('well', 4036.568083135732), ('smart', 3488.733400902889), ('buy', 3302.4942106337057), ('works', 3285.196633491496)], 'Topic 1:': [('antenna', 10912.485909226147), ('channels', 8085.649643923279), ('cable', 6581.637832312293), ('get', 3967.363126317774), ('signal', 3893.334433241468), ('stations', 2579.2760667453376), ('digital', 2171.439119653906), ('reception', 2006.8055614095251), ('local', 1810.5481909990851), ('channel', 1774.5743551758446)], 'Topic 2:': [('mount', 10947.629996696167), ('wall', 8927.870948624342), ('easy', 5132.73864743966), ('would', 3010.9979333188444), ('well', 2878.188525087894), ('install', 2849.5555573843862), ('price', 2437.7119428302894), ('mounting', 2299.225657565528), ('mounted', 2214.4119562251803), ('product', 2164.3658917856746)], 'Topic 3:': [('sound', 11224.48

In [31]:
lda.display_topic_model()

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [ ]:
for name, group in df_prod.groupby('product'):
    count = group.count()
    if (count[1] > 100):
        lda = LdaReviewAnalyzer(df_prod)
        lda.fit_transform(num_topics=2, rating=[0,1])
        topics_dict = lda.get_topics()
        lda.display_topic_model()
        print (name)
        print (topics_dict)
        print ("____________")

In [ ]:
from src.w2v_review_analyzer import W2VReviewAnalyzer
import pandas as pd
df = pd.DataFrame()
w2v = W2VReviewAnalyzer(df)
# w2v.fit()

In [ ]:
import pickle
#pickle.dump( w2v.model, open( f'static/w2v.pkl', "wb" ) )
w2v.model = pickle.load(open( f'static/w2v.pkl', "rb" ) )

In [ ]:
w2v.model.wv.vocab


In [ ]:
#w2v.model.wv.most_similar(positive=['samsung',['screen'])

samsung_vec = w2v.model.wv.get_vector('samsung')
broken_vec = w2v.model.wv.get_vector('broken')

w2v.model.wv.most_similar(positive=['ln46c630','cost'])

#w2v.model.wv.similarity('defective','samsung')

In [ ]:
import tensorflow as tf
import os
from tensorflow.contrib.tensorboard.plugins import projector
import warnings
warnings.filterwarnings("ignore")

def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 100))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [ ]:

visualize(w2v.model.wv, 'data/tensor/')

In [ ]:
l = [{'a':4},{'b':2},{'c':3}]

In [ ]:
sorted(l)